In [16]:
import Pkg
Pkg.add("DifferentialEquations")
Pkg.add("Distributions")
Pkg.add("Turing")
Pkg.add("DiffEqBayes")
Pkg.add("ParameterizedFunctions")
Pkg.add("RecursiveArrayTools")

 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
  [731186ca] + RecursiveArrayTools v2.1.0
  Updating `~/.julia/environments/v1.3/Manif

In [10]:
using DifferentialEquations
using DiffEqBayes
using ParameterizedFunctions
f1 = @ode_def begin
  dx = a*x - b*x*y
  dy = -c*y + d*x*y
end a b c d
p = [1.5,1.0,3.0,1.0]
u0 = [1.0,1.0]
tspan = (0.0,10.0)
prob1 = ODEProblem(f1,u0,tspan,p)
sol = solve(prob1,Tsit5())
t = collect(range(1,stop=10,length=10))
randomized = VectorOfArray([(sol(t[i]) + .01randn(2)) for i in 1:length(t)])
data = convert(Array,randomized)

2×10 Array{Float64,2}:
 2.77065   6.77585  0.979398  1.88623   …  4.3439    3.22968  1.05103 
 0.267786  2.01844  1.91243   0.330993     0.309853  4.55666  0.898765

In [14]:
pf_func = function (du,u,p,t)
  du[1] = p[1] * u[1] - p[2] * u[1]*u[2]
  du[2] = -3 * u[2] + u[1]*u[2]
end

u0 = [1.0;1.0]
tspan = (0.0,10.0)
p = [1.5,1.0]
pg_func = function (du,u,p,t)
  du[1] = 1e-6u[1]
  du[2] = 1e-6u[2]
end
prob = SDEProblem(pf_func,pg_func,u0,tspan,p)
sol = solve(prob,SRIW1())

retcode: Success
Interpolation: 1st order linear
t: 565-element Array{Float64,1}:
  0.0                 
  0.011713181719699725
  0.01405581806363967 
  0.01669128395057211 
  0.0196561830733711  
  0.022991694586519968
  0.026744145038812443
  0.030965651797641477
  0.03571484690132414 
  0.04105769139296714 
  0.04706839144606551 
  0.053830429005801185
  0.061437721260503814
  ⋮                   
  9.78472527627678    
  9.804446250307329   
  9.824462256834902   
  9.844787365062608   
  9.865436405753467   
  9.886424933351986   
  9.907769132672604   
  9.929485734262661   
  9.951591864779692   
  9.974104915087294   
  9.997042292960321   
 10.0                 
u: 565-element Array{Array{Float64,1},1}:
 [1.0, 1.0]                              
 [1.0060114608191897, 0.9768817431109275]
 [1.00725049704347, 0.9723306421936018]  
 [1.008658972695803, 0.9672393257242939] 
 [1.010262047105421, 0.9615478254793435] 
 [1.0120889374398112, 0.9551902196642447]
 [1.0141737172366572, 0.94

In [17]:
using RecursiveArrayTools # for VectorOfArray
t = collect(range(0, stop=10, length=200))
function generate_data(t)
  sol = solve(prob,SRIW1())
  randomized = VectorOfArray([(sol(t[i]) + .01randn(2)) for i in 1:length(t)])
  data = convert(Array,randomized)
end
aggregate_data = convert(Array,VectorOfArray([generate_data(t) for i in 1:10000]))

2×200×10000 Array{Float64,3}:
[:, :, 1] =
 0.997587  1.0343    1.05549   1.11195   …  0.970138  1.00042  1.02059 
 1.00707   0.904328  0.813926  0.753695     1.09637   1.00621  0.922412

[:, :, 2] =
 1.01848  1.03319   1.05163   1.10937   …  0.971732  0.983219  1.0275  
 1.01005  0.912574  0.823675  0.745682     1.11926   1.02322   0.927973

[:, :, 3] =
 1.00908  1.03486   1.0666    1.11408   …  0.976667  0.996377  1.04066 
 1.01085  0.909984  0.813516  0.755824     1.13321   1.00299   0.908734

...

[:, :, 9998] =
 0.993568  1.0231    1.0561    1.10903   …  0.974645  0.967264  1.0318  
 1.01143   0.906217  0.796896  0.760925     1.12446   1.00199   0.908593

[:, :, 9999] =
 1.00569  0.995034  1.06941   1.08808   …  0.966421  1.01131  1.02528 
 1.00553  0.888381  0.833385  0.747837     1.11895   1.02403  0.916534

[:, :, 10000] =
 1.0113  1.03626   1.06953   1.08652   …  0.9834   0.965656  1.00204  1.01961
 1.002   0.919041  0.825666  0.732445     1.22414  1.1022    1.0342   0.90165

In [11]:
using Distributions
priors = [Truncated(Normal(1.5,0.1),0,2),Truncated(Normal(1.0,0.1),0,1.5),
          Truncated(Normal(3.0,0.1),0,4),Truncated(Normal(1.0,0.1),0,2)]

4-element Array{Truncated{Normal{Float64},Continuous,Float64},1}:
 Truncated(Normal{Float64}(μ=1.5, σ=0.1), range=(0.0, 2.0))
 Truncated(Normal{Float64}(μ=1.0, σ=0.1), range=(0.0, 1.5))
 Truncated(Normal{Float64}(μ=3.0, σ=0.1), range=(0.0, 4.0))
 Truncated(Normal{Float64}(μ=1.0, σ=0.1), range=(0.0, 2.0))

In [18]:
using Turing
bayesian_result = turing_inference(prob,Tsit5(),t,aggregate_data,priors;num_samples=500)

BoundsError: BoundsError: attempt to access 2×200×10000 Array{Float64,3} at index [Base.Slice(Base.OneTo(2)), 1]